# Large Language Models: Hugging Face

Vamos a utilizar la biblioteca transformers de Hugging Face para esto, ya que proporciona herramientas tanto para tokenizar como para decodificar texto, adecuadas para LLMs.

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


c:\Users\icira\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Definimos un prompt de entrada. Este texto es el que queremos enviar al modelo. En un caso de uso real, este texto podría ser una pregunta, una declaración o cualquier entrada que desees procesar con el modelo.

In [2]:
prompt = "The future of artificial intelligence is"

Creamos una instancia del tokenizer utilizando un modelo pre-entrenado de GPT-2. El tokenizer es responsable de convertir el texto en una forma que el modelo puede entender (tokens). Los modelos de lenguaje como GPT-2, GPT-3 y GPT-4 trabajan con tokens, no con texto crudo.

A continuación usamos el método encode para convertir el texto en tokens. El método encode devuelve un objeto de tipo BatchEncoding, que contiene los tokens codificados y otra información útil.

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input_tokens = tokenizer.encode(prompt, return_tensors="pt")
print("Vector de tokens:", input_tokens)

Vector de tokens: tensor([[  464,  2003,   286, 11666,  4430,   318]])


Usamos un modelo pre-entrenado de GPT-2 para generar texto. El modelo toma los tokens codificados como entrada y devuelve un objeto de tipo BaseModelOutput, que contiene los tokens decodificados y otra información útil.

In [4]:
import torch 

# Configuramos el dispositivo para usar GPU si está disponible, de lo contrario CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [6]:
model.to(device)
input_tokens = input_tokens.to(device)

In [7]:
# Generamos la salida con el modelo
output_tokens = model.generate(input_tokens, max_length=10, num_return_sequences=1)
print("Tokens de respuesta:", output_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tokens de respuesta: tensor([[  464,  2003,   286, 11666,  4430,   318,  8627,    13,   198,   198]])


In [ ]:
[  464,  2003,   286, 11666,  4430,   318]

Usamos el método decode del tokenizer para convertir los tokens de respuesta de nuevo a texto. Este paso es el inverso de la tokenización y nos permite convertir la salida del modelo (en forma de tokens) de nuevo a un formato legible por humanos.

In [10]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("Texto generado:", generated_text)

Texto generado: The future of artificial intelligence is uncertain.




# Large Language Models: API de OpenAI

Importamos las librerías necesarias, como vamos a trabajar con OpenAI, una vez instalado el paquete, necesitamos configurar nuestra clave para la API.

Pueden obtenerse claves en la plataforma de configuración de OpenAI en [este enlace](https://platform.openai.com/api-keys).

In [11]:
import openai

# Método 1: Habiendo configurado la variable de entorno OPENAI_API_KEY
#import os
#openai.api_key = os.environ["OPENAI_API_KEY"]

# Método 2: Importando la variable de un archivo, la idea es no sincronizar este archivo con el repositorio para proteger la clave
from keys import openai_api_key
openai.api_key = openai_api_key

# Método 3: Ingresando la clave directamente
#openai.api_key = "sk-31313142142112312312123123123123"

Definimos una función para lanzar "prompts" a la API de OpenAI. Esta función toma como entrada un prompt, el modelo a utilizar y el número de tokens máximo a generar; y devuelve la respuesta de la API.

La lista de modelos disponibles se puede encontrar en [este enlace](https://platform.openai.com/docs/models/continuous-model-upgrades).

In [12]:
def query_gpt(prompt, model="text-davinci-003", max_tokens=100):
    # Realiza la solicitud a la API
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens
    )

    # Devuelve el texto de la respuesta
    return response.choices[0].text.strip()

In [14]:
# Ejemplo de uso
prompt = "¿Cuáles son las ventajas de los modelos de lenguaje grande como GPT-4?"
respuesta = query_gpt(prompt, max_tokens=1000)
print("Respuesta:\n", respuesta)


Respuesta:
 1. Permite el aprendizaje profundo y la generación de texto autónomo de forma fácil y flexible.
2. Proporciona una variedad de contenido más rica y variada.
3. Morfología bien integrada para mejorar la comprensión del texto.
4. Mejorada capacidad de comprender el lenguaje natural.
5. Capacidad de trabajar con lenguaje libre y no modelado.
6. Capacidad de predecir correctamente el contexto del discurso.
7. Mayor precisión que modelos más pequeños.
8. Mejor detección de relaciones y patrones en los textos.
9. Mayor diversidad en los temas que pueden abordarse.
10. Amigable para el desarrollador y fácil de implementar.


# ChatBot

Podemos encapsular el código anterior en una clase para crear un chatbot. Esta clase tiene un método llamado chat, que toma como entrada un prompt y devuelve la respuesta del modelo. Además, la clase guarda la historia de la conversación para que los nuevos prompts puedan ser construidos a partir del contexto que se ha generado hasta el momento.

In [15]:
import openai
from keys import openai_api_key
openai.api_key = openai_api_key

In [16]:
class ChatBot:
    def __init__(self, model, max_tokens, max_history):
        self.model = model
        self.history = []
        self.max_tokens = max_tokens
        self.max_history = max_history

    def chat(self, prompt):

        messages = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]

        for i in range(len(self.history)):
            messages.append({"role": "user", "content": self.history[i][0]})
            messages.append({"role": "assistant", "content": self.history[i][1]})
            
        messages.append({"role": "user", "content": prompt})

        response = openai.ChatCompletion.create(
            model = self.model,
            messages = messages
        )

        # Devuelve el texto de la respuesta
        rsp = response.choices[0]["message"]["content"]

        self.history.append((prompt, rsp))
        if len(self.history) > self.max_history:
            self.history = self.history[1:]

        return rsp

Creamos un bucle while para que el chatbot pueda mantener una conversación con el usuario. El bucle se ejecuta hasta que el usuario escribe "bye".

In [17]:
chatbot = ChatBot(model="gpt-3.5-turbo", max_tokens=100, max_history=3)

prompt = ""
while prompt != "bye":
    prompt = input("You: ")
    print("User:    " + prompt)
    response = chatbot.chat(prompt)
    print("Chatbot: " + response)
    print()


User:    hello
Chatbot: Hello! How can I assist you today?

User:    how are you?
Chatbot: As an AI assistant, I don't have feelings, but I'm here and ready to assist you with whatever you need. How can I help you today?

User:    
Chatbot: If you have any questions or need assistance with anything, feel free to let me know. I'm here to help!



KeyboardInterrupt: Interrupted by user

# LangChain: Procesado de documentos

LangChain es una herramienta de procesamiento de lenguaje natural que permite a los usuarios procesar documentos de texto. La herramienta se puede utilizar para realizar tareas como la tokenización, la lematización, la extracción de entidades, la clasificación de texto, la traducción automática y la generación de texto.

En este caso, vamos a usar LangChain junto con una Vector Database (ChromaDB), que es una base de datos diseñada para almacenar los textos tokenizados de forma eficiente. La base de datos se puede utilizar para realizar búsquedas de texto, recuperar documentos similares y realizar otras tareas de procesamiento de texto.

In [18]:
import openai
from keys import openai_api_key
openai.api_key = openai_api_key

In [19]:
from langchain.document_loaders import PyPDFLoader 

from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

In [21]:
pdf_path = "data/AttentionIsAllYouNeed.pdf"      

loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents, 
                                 embedding=embeddings, 
                                 persist_directory="db")
vectordb.persist()
memory = ConversationBufferMemory(memory_key="chat_history", 
                                  return_messages=True)

pdf_qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0.8, model_name="gpt-3.5-turbo-16k"), 
    vectordb.as_retriever(search_kwargs={'k': 4}),
    return_source_documents=True,
    memory=memory)


In [22]:
def qa_history(query, chat_history = []):
        pdf_qa = ConversationalRetrievalChain.from_llm(
            ChatOpenAI(temperature=0.8, model_name="gpt-3.5-turbo-16k"), 
            vectordb.as_retriever(search_kwargs={'k': 4}),
            #self.vectordb.as_retriever(), 
            return_source_documents=True)
                
        print("Q: " + query)
                
        result = pdf_qa({"question": query, "chat_history": chat_history})
        answer = result['answer']
        
        print("A: " + answer)
        chat_history.append((query, answer))
        return answer, chat_history

In [23]:
chat_history = []
while True:
    query = input("Q: ")
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        import sys
        sys.exit()
    response, chat_history = qa_history(query, chat_history)
    print("")

Q: who is the author of the document?
A: The author of the document is not mentioned in the given context.

Q: summarize the context
A: The context of the document is a discussion about the Law and its application, with the opinion that while the Law may never be perfect, its application should be just. The document also includes figures showing attention heads in layer 5 of 6, apparently involved in anaphora resolution and exhibiting behavior related to the structure of the sentence.

Q: conclusions of the text?
A: The conclusions of the text are not clear from the given context.

Q: 


KeyboardInterrupt: 